# No going back from here...

In [ ]:
from bbq.field import Field
from bbq.polynomial import Monomial
from bbq.bbq_code import BivariateBicycle
from bbq.circuit import construct_sm_circuit, generate_noisy_circuit, simulate_x_circuit, generate_single_error_circuits, build_hx_dict, build_hz_dict, build_hx_eff, build_hz_eff, construct_decoding_matrix

import numpy as np
import matplotlib.pyplot as plt
import pickle
from scipy.sparse import coo_matrix, hstack
from ldpc import BpOsdDecoder


In [ ]:
with open('./ult', 'rb') as fp:
	bravyi = pickle.load(fp)


In [ ]:
# Set up
x_order = ['idle', 1, 4, 3, 5, 0, 2]
z_order = [3, 5, 0, 1, 2, 4, 'idle']

p = 0.003
num_cycles = 2
error_rates = {'Meas': p, 'Prep': p, 'idle': p, 'CNOT': p}


In [ ]:
# Construct parity check matrices
field = Field(2)
x, y = Monomial(field, 'x'), Monomial(field, 'y')
a = x**3 + y + y**2
b = y**3 + x + x**2
bb = BivariateBicycle(a, b, 6, 6, 1)
hx, lx = bb.hx, bb.x_logicals
hz, lz = bb.hz, bb.z_logicals
n_qudits = hx.shape[1]

bb.A = [bb.A[2], bb.A[0], bb.A[1]]  # reorder to match bravyi
bb.edges = bb._edges()


In [ ]:
bb.x_logicals = np.array([[0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]])
bb.z_logicals = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]])
lx = bb.x_logicals
lz = bb.z_logicals


In [ ]:
np.all(hx == bravyi['hx']), np.all(hz == bravyi['hz'])


(np.True_, np.True_)

In [ ]:
bb.data_qudits == bravyi['data_qubits'], bb.Xchecks == bravyi['Xchecks'], bb.Zchecks == bravyi['Zchecks']


(True, True, True)

In [ ]:
# Remove qudit part
bb.edges
nbs = {}
for edge in bb.edges:
    nbs[edge] = bb.edges[edge][0]


In [ ]:
nbs == bravyi['nbs']


True

In [ ]:
circ = construct_sm_circuit(bb, x_order, z_order)


In [ ]:
cycle = []
for gate in circ:
    if gate[0] == 'CNOT':
        cycle.append((gate[0], gate[1], gate[2]))
    else:
        cycle.append(gate)


In [ ]:
cycle == bravyi['cycle']


True

In [ ]:
x_prob, x_circuit, z_prob, z_circuit = generate_single_error_circuits(circ * num_cycles, error_rates)


In [ ]:
x_circs = []
for circuit in x_circuit:
    c = []
    for gate in circuit:
        if gate[0] == 'CNOT':
            c.append((gate[0], gate[1], gate[2]))
        else:
            c.append(gate)
    x_circs.append(c)

z_circs = []
for circuit in z_circuit:
    c = []
    for gate in circuit:
        if gate[0] == 'CNOT':
            c.append((gate[0], gate[1], gate[2]))
        else:
            c.append(gate)
    z_circs.append(c)


In [ ]:
x_circs == bravyi['circuitsX'], z_circs == bravyi['circuitsZ']


(True, True)

In [ ]:
x_prob == bravyi['ProbX'], z_prob == bravyi['ProbZ']


(True, True)

In [ ]:
hx_dict = build_hx_dict(bb, x_circuit, circ, num_cycles)
hz_dict = build_hz_dict(bb, z_circuit, circ, num_cycles)


In [ ]:
hx_dict == bravyi['HXdict'], hz_dict == bravyi['HZdict']


(True, True)

In [ ]:
hx_eff, short_hx_eff, channel_prob_x = build_hx_eff(bb, hx_dict, x_prob, num_cycles)
hz_eff, short_hz_eff, channel_prob_z = build_hz_eff(bb, hz_dict, z_prob, num_cycles)


In [ ]:
np.all(hx_eff == bravyi['HX'].toarray()), np.all(hz_eff == bravyi['HZ'].toarray())


(np.True_, np.True_)

In [ ]:
np.all(short_hx_eff == bravyi['HdecX'].toarray()), np.all(short_hz_eff == bravyi['HdecZ'].toarray())


(np.True_, np.True_)

In [ ]:
np.all(channel_prob_x == bravyi['channel_probsX']), np.all(channel_prob_z == bravyi['channel_probsZ'])


(np.True_, np.True_)

In [ ]:
physical_error = np.logspace(-1.7, -3, 10)
results = []
print(f'Starting full trials over physical error rates {physical_error}')

for p in physical_error:
    error_rates = {'Meas': p, 'Prep': p, 'idle': p, 'CNOT': p}
    fails = 0
    trials = 0

    # Generate sm circuit and decoding matrix
    hx_eff, short_hx_eff, hz_eff, short_hz_eff, channel_prob_x, channel_prob_z = construct_decoding_matrix(bb, circ, error_rates, num_cycles)

    while fails < 10:
        trials += 1
        
        # Generate noisy circ
        noisy_circ, err_cnt = generate_noisy_circuit(bb, circ * num_cycles, error_rates)
        
        # Simulate noisy circ
        x_syndrome_history, x_state, x_syndrome_map, x_err_count = simulate_x_circuit(bb, noisy_circ + circ + circ)

        # Calculate true logical effect
        qudits_dict = bb.qudits_dict
        data_qudits = bb.data_qudits
        x_state_data_qudits = [x_state[qudits_dict[qudit]] for qudit in data_qudits]
        x_syndrome_final_logical = (np.array(lz) @ x_state_data_qudits) % field.p
        
        # Syndrome sparsification
        z_checks = bb.Zchecks
        x_syndrome_history_copy = x_syndrome_history.copy()
        for check in z_checks:
            pos = x_syndrome_map[check]
            assert len(pos) == num_cycles + 2
            for row in range(1, num_cycles + 2):
                x_syndrome_history[pos[row]] += x_syndrome_history_copy[pos[row-1]]
        x_syndrome_history %= field.p
        
        # Decode
        bposd = BpOsdDecoder(short_hx_eff, error_channel=channel_prob_x, max_iter=1000)
        x_error = bposd.decode(x_syndrome_history)
        assert np.all((short_hx_eff @ x_error) % field.p == x_syndrome_history)
        
        # Check logical effect
        first_logical_row = bb.l * bb.m * (num_cycles + 2)
        k = len(lx)
        x_syndrome_history_augmented_guessed = (hx_eff @ x_error) % field.p
        x_syndrome_final_logical_guessed = x_syndrome_history_augmented_guessed[first_logical_row: first_logical_row + k]
        
        if not np.array_equal(x_syndrome_final_logical_guessed, x_syndrome_final_logical):
            fails += 1
            print(f'Physical error rate: {p:.4f}, Trial: {trials}, Failures: {fails}')
    print('--------------------------------------------------------------')
    results.append(trials)


Starting full trials over physical error rates [0.01995262 0.01430723 0.01025914 0.00735642 0.005275   0.00378249
 0.00271227 0.00194486 0.00139458 0.001     ]
Physical error rate: 0.0200, Trial: 1, Failures: 1
Physical error rate: 0.0200, Trial: 2, Failures: 2
Physical error rate: 0.0200, Trial: 4, Failures: 3
Physical error rate: 0.0200, Trial: 5, Failures: 4
Physical error rate: 0.0200, Trial: 6, Failures: 5
Physical error rate: 0.0200, Trial: 7, Failures: 6
Physical error rate: 0.0200, Trial: 8, Failures: 7
Physical error rate: 0.0200, Trial: 9, Failures: 8
Physical error rate: 0.0200, Trial: 10, Failures: 9
Physical error rate: 0.0200, Trial: 12, Failures: 10
--------------------------------------------------------------
Physical error rate: 0.0143, Trial: 3, Failures: 1
Physical error rate: 0.0143, Trial: 4, Failures: 2
Physical error rate: 0.0143, Trial: 6, Failures: 3
Physical error rate: 0.0143, Trial: 7, Failures: 4
Physical error rate: 0.0143, Trial: 11, Failures: 5
Physical

In [ ]:
results


NameError: name 'results' is not defined

In [ ]:
plt.plot(physical_error, 10/(2*np.array(results)), '-.', label='3x3', color='crimson', marker='o')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Physical error rate')
plt.ylabel('Logical error rate')
plt.legend()
plt.title('Threshold Graph of 72-Qubit Bivariate Bicycle Code');
